In [1]:
import numpy as np

In [2]:
def entropy(y):
    hist = np.bincount(y)
    ps = hist / len(y)
    return -np.sum([p * np.log2(p) for p in ps if p > 0])

In [3]:
class Node:
    def __init__(self, feature=None, left=None, right=None, threshold=None, value=None):
        self.feature = feature
        self.left = left
        self.right = right
        self.threshold = threshold
        self.value = value
        
    def is_leaf_node(self):
        return self.value is not None

In [4]:
class DecisionTree:
    def __init__(self, min_sample_split=2, max_depth=100, n_feature=None):
        self.n_feature = n_feature
        self.min_sample_split = min_sample_split
        self.max_depth = max_depth
        
    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.n_feature = X.shape[1] if not self.n_feature else min(self.n_feature, n_features)
        self.root = self.growTree(X, y)
        
    def predict(self, X):
        ypred = [self.traverseTree(x, self.root) for x in X]
        return np.array(ypred)
    
    def growTree(self, X, y, depth=0):
        n_samples, n_features = X.shape
        n_labels = np.unique(y)
        
        if (depth >= max_depth or n_samples < self.min_sample_split 
           or n_labels == 1):
            leaf_value = self.most_common_label(y)
            return Node(value=leaf_value)
        
        
        feature_idxs = np.random.choice(n_features, self.n_feature, replace=False)
        bestFeature, bestThreshold = self.bestCriteria(X, y, feature_idxs)
        
        left_idxs, right_idxs = self.split(X[:, bestFeature], bestThreshold)
        left = self.growTree(X[left_idxs, :], y[left_idxs], depth+1)
        right = self.growTree(X[right_idxs, :], y[right_idxs], depth+1)
        
        return Node(feature=bestFeature, left = left, right=right, threshold=bestThreshold)
    
    def bestCriteria(X, y, feature_idxs):
        best_gain = -1
        splitIdx, splitThreshold = None, None
        
        for f in feature_idxs:
            X_column = X[:, f]
            thresholds = np.unique(X_column)
            for t in thresholds:
                gain = self.infoGain(X_column, y, t)
                
                if gain > best_gain:
                    best_gain = gain
                    split_idx = f
                    split_thresh = t
            
        return split_idx, split_thresh
            
    def infoGain(self, X, y, t):
        pE = entropy(y)
        
        leftIdx, rightIdx = self.split(X, t)
        if len(leftIdx)==0 or len(rightIdx)==0:
            return 0
        
        n = len(y)
        n_l, n_r = len(leftIdx), len(rightIdx)
        e_l, e_r = entropy(y[leftIdx]), entropy(y[rightIdx])
        
        child_entropy = (n_l * e_l) / n + (n_r * e_r) / n
        ig = pE - child_entropy
        return ig
    
    def split(self, X, threshold):
        leftIdxs = np.argwhere(X>= threshold).flatten()
        rightIdxs = np.argwhere(X<threshold).flatten()
        return leftIdxs, rightIdxs
    
    def traverseTree(self, X, node):
        if node.is_leaf_node():
            return node.value
        
        if X[node.feature] <= node.threshold:
            return traverseTree(X, node.left)
        return traverseTree(X, node.right)
    
    def most_common_label(self, label):
        mostCommon = Counter(label)
        mc = mostCommon(1)[0][0]
        return mc